# Langevin dynamics of a polymer attached to a plane

In [ ]:
%load_ext autoreload
%autoreload 2
# %matplotlib inline

In [ ]:
import hoomd
import gsd.hoomd
import numpy as np
import matplotlib.pyplot as plt

## Functions

In [ ]:
# This is not intended as a full tutorial on fresnel - see the fresnel user
# documentation (https://fresnel.readthedocs.io/) if you would like to learn more.

import io
import warnings

import fresnel
import IPython
import packaging.version
import PIL

device = fresnel.Device()
tracer = fresnel.tracer.Path(device=device, w=300, h=300)

FRESNEL_MIN_VERSION = packaging.version.parse('0.13.0')
FRESNEL_MAX_VERSION = packaging.version.parse('0.14.0')


def render(snapshot, formovie=False):
    import math
    if (
        'version' not in dir(fresnel)
        or packaging.version.parse(fresnel.version.version) < FRESNEL_MIN_VERSION
        or packaging.version.parse(fresnel.version.version) >= FRESNEL_MAX_VERSION
    ):
        warnings.warn(
            f'Unsupported fresnel version {fresnel.version.version} - expect errors.'
        )
    L = snapshot.configuration.box[0]
    scene = fresnel.Scene(device)
    geometry = fresnel.geometry.Sphere(
        scene, N=len(snapshot.particles.position), radius=0.5
    )
    geometry.material = fresnel.material.Material(
        color=fresnel.color.linear([252 / 255, 209 / 255, 1 / 255]), roughness=0.5
    )
    geometry.material.primitive_color_mix = 1
    geometry.position[:] = snapshot.particles.position[:]
    geometry.outline_width = 0.08
    typeid = snapshot.particles.typeid
    geometry.color[typeid == 1, :] = fresnel.color.linear(
        [90 / 255, 226 / 255, 75 / 255]
    )
    geometry.color[typeid == 0, :] = fresnel.color.linear(
        [252 / 255, 209 / 255, 1 / 255]
    )
    fresnel.geometry.Box(scene, [L, L, L, 0, 0, 0], box_radius=0.02)

    scene.lights = [
        fresnel.light.Light(direction=(0, 0, 1), color=(0.8, 0.8, 0.8), theta=math.pi),
        fresnel.light.Light(
            direction=(1, 1, 1), color=(1.1, 1.1, 1.1), theta=math.pi / 3
        ),
    ]
    scene.camera = fresnel.camera.Orthographic(
        position=(L * 2, L, L * 2), look_at=(0, 0, 0), up=(0, 1, 0), height=L * 1.4 + 1
    )
    scene.background_alpha = 1
    scene.background_color = (1, 1, 1)
    if formovie:
        return tracer.sample(scene, samples=500)
    else:
        return IPython.display.Image(tracer.sample(scene, samples=500)._repr_png_())

def render_movie(frames):
    a = render(frames[0], formovie=True)

    im0 = PIL.Image.fromarray(a[:, :, 0:3], mode='RGB').convert(
        'P', palette=PIL.Image.Palette.ADAPTIVE
    )
    ims = []
    for i, f in enumerate(frames[1:]):
        a = render(f, formovie=True)
        im = PIL.Image.fromarray(a[:, :, 0:3], mode='RGB')
        im_p = im.quantize(palette=im0)
        ims.append(im_p)

    blank = np.ones(shape=(im.height, im.width, 3), dtype=np.uint8) * 255
    im = PIL.Image.fromarray(blank, mode='RGB')
    im_p = im.quantize(palette=im0)
    ims.append(im_p)

    f = io.BytesIO()
    im0.save(f, 'gif', save_all=True, append_images=ims, duration=100, loop=0)

    size = len(f.getbuffer()) / 1024
    if size > 3000:
        warnings.warn(f'Large GIF: {size} KiB')
    return IPython.display.display(IPython.display.Image(data=f.getvalue()))

## Construct the simulation

We simulate the dynamics of a polymer chain tethered to a wall. The chain is made of $N$ monomers. Let us $r_i$ be the coordinates of the $i$th monomer, an $u_i = r_{i} - r_{i-1}$ the $i$th bond vector. We will consider a chain with the following potential energy:
1. FENE bonds
\begin{aligned}
U_\text{FENE} = -\frac{3 k_\text{e} r_0^2}{2 b^2} \sum \limits_{i=1}^{N}\log{\left(1 -  \frac{u_i^2}{r_0^2}\right)}.
\end{aligned}

2. Kratky-Porod angle interactions
\begin{aligned}
U_\text{KP} = l_\text{p} \sum \limits_{i=1}^{N-1} (1 - \cos{\left( [u_i ; u_{i+1}] \right)}.
\end{aligned}

3. Truncated Lennard-Jones potential
\begin{aligned}
U_\text{LJ} = 4 \epsilon \left( \left( \frac{\sigma}{r} \right)^{12} - \left( \frac{\sigma}{r} \right)^{6}\right), \quad \text{if} \quad r < r_\text{cut}
\end{aligned}

Parameters

In [ ]:
# model
N_monomer = 2**5   # number of monomers
lp = 3.            # bending rigidity

# simulation
seed = 123
itermax = int(1e4)
idump = int(1e2)
dt = 0.005

We start by defining the dimensions of the simulation box. By default, there are periodic boundary conditions, so coordinates are warped around when a particle crosses one of the box boundaries. Here we want the box to be long enough so that a chain tethered at the center will not cross one of the box boundaries.

In [ ]:
Lx = 2*N_monomer
Ly = Lx
Lz = Lx

We generate an initial configuration for the coordinates of the polymer. The easiest is a straight chain, i.e. $u_i = (0,0,1)$.

In [ ]:
# polymer(s)
position_polymer = position = np.concatenate([np.zeros((N_monomer,2)), (np.arange(N_monomer)+1).reshape(-1,1)],axis=1)

We will also model an impenetrable membrane. Here we take the approach of representing the membrane as a sheet of closely packed spheres. This approach can be used to represent more complex shapes.

We generate a grid of coordinates in the $xy$ plane at $z=0$.

In [ ]:
# barrier (e.g. plane)
xvalues = np.linspace(-Lx/2, Lx/2, max(1,int(np.ceil(Lx/1.))), endpoint=False)
yvalues = np.linspace(-Ly/2, Ly/2, max(1,int(np.ceil(Ly/1.))), endpoint=False)

X = np.einsum('i,j->ij', xvalues, np.ones(xvalues.shape)).ravel().reshape(-1,1)
Y = np.einsum('j,i->ij', yvalues, np.ones(xvalues.shape)).ravel().reshape(-1,1)
Z = np.zeros(X.shape)

position_wall = np.concatenate([X,Y,Z], axis=1)

In HOOMD, a simulation starts from a state. A state can be defined through the `Frame` object.

We start by:
* specifying the number of particles
* declaring particle types that are convenient or relevant
* setting the dimensions of the box

In our case, we will differentiate the polymer particles from the barrier/membrane particle.

In [ ]:
frame = gsd.hoomd.Frame()
frame.particles.N = len(position_polymer) + len(position_wall)
frame.particles.types = ['mobile','barrier']
frame.configuration.box = [Lx, Ly, Lz, 0, 0, 0]

We now set:
* the positions of the particles, using the initial values computed above
* set their type ID

In [ ]:
frame.particles.position = np.concatenate([position_polymer, position_wall])
frame.particles.typeid = np.concatenate([[1] + [0]*(len(position_polymer)-1) + [1]*len(position_wall)])

Now we need to specify the type of bonded interactions. To do so, we:
* specify the number of bonds
* declare relevant bond types that we will be using: in our case just one

Then for each of the $N-1$, bonds we indicate:
* the index of the type of bond to implement there, in our case it is always 0.
* the pair of monomers, $(i-1,i)$, which are bonded

In [ ]:
# set up the bonded interactions
frame.bonds.N = N_monomer - 1
frame.bonds.types = ['A-A']
frame.bonds.typeid = [0] * (N_monomer - 1)
frame.bonds.group = np.array([np.arange(0,N_monomer-1), np.arange(1,N_monomer)]).T

# we use a FENE potential
fene = hoomd.md.bond.FENEWCA()
fene.params['A-A'] = dict(k=20., r0=1.5, epsilon=1., sigma=1., delta=0.)

The Kratky-Porod potential defined earlier to model bending rigidity amounts to specify so-called angle interactions. Each angle interaction involves 3 monomers. The implementation is similar to the implementation of bonded interactions, except that we are now specifying triplets $(i-1,i,i+1)$ of monomers for each angle.

The Kratky-Porod potential is not implemented in HOOMD. However, we can use the `Table` method to implement our custom potential. We supply a tabular estimate of our potential for a finite number of values for the angle.

In [ ]:
# set the angle interactions
frame.angles.N = N_monomer - 2
frame.angles.types = ['A-A-A']
frame.angles.typeid = [0] * (N_monomer - 2)
frame.angles.group = np.array([np.arange(0,N_monomer-2), np.arange(1,N_monomer-1), np.arange(2,N_monomer)]).T

# set the angle interactions as a Kratky-Porod potential
width = 2**7  # Number of points in the table
theta_values = np.linspace(0, np.pi, width)

## Calculate the potential V(theta) using the Kratky-Porod model
V_theta = 0.5 * lp * (1 - np.cos(theta_values-np.pi))

## Calculate the torque tau(theta) as the derivative of the potential
tau_theta = -0.5 * lp * np.sin(theta_values-np.pi)
kratky = hoomd.md.angle.Table(width=width)
kratky.params['A-A-A'] = dict(U=V_theta, tau=tau_theta)

The last type of interactions considered are pair interactions. For each pair of monomer, the interaction potential $U(r_i - r_j)$ is used to compute the force exerted by monomer $j$ on $i$ (and vice-versa). Here we use truncated Lennard-Jones potentials. We need to specify an energy scale, $\epsilon$, and a range for the interaction, $r_\text{cut}$. When $r_\text{cut} = 2^{1/6} \sigma$, the interaction is purely repulsive, which is what we consider here. For larger $r_\text{cut}$, there is also an attractive region in the potential.

In [ ]:
# excluded volume interactions
lj = hoomd.md.pair.LJ(nlist=hoomd.md.nlist.Cell(buffer=0.4, exclusions=('bond','angle')))
lj.params[('mobile', 'mobile')] = dict(epsilon=1.0, sigma=1.0)
lj.r_cut[('mobile', 'mobile')] = 2**(1 / 6)

lj.params[('mobile', 'barrier')] = dict(epsilon=1.0, sigma=1.0)
lj.r_cut[('mobile', 'barrier')] = 2 ** (1 / 6)

# the particle in the membrane will remain immobile so we do not need
# to implement excluded volume interactions among them
lj.params[('barrier', 'barrier')] = dict(epsilon=0.0, sigma=0.0)
lj.r_cut[('barrier', 'barrier')] = 0

Let us look at what we constructed

In [ ]:
render(frame)

## Run the simulation

We have successfully prepared an initial state. Now all there is to do is to integrate the equations of motion.

In [ ]:
cpu = hoomd.device.CPU()    # run on the CPU
simulation = hoomd.Simulation(device=cpu, seed=seed) # create the simulation object

We need to set the simulation state to what we have created previously

In [ ]:
simulation.create_state_from_snapshot(frame)

Now we specify the type of dynamics, or "integrator", that we will use. We will use the Langevin dynamics integrator. As mentioned hereabove, the particles which make the barrier are fixed, so we only integrate the equation of motion for the mobile particles.

We also need to add to the integrator the different force fields that we have created.

In [ ]:
langevin = hoomd.md.methods.Langevin(filter=hoomd.filter.Type(types=['mobile']), kT=1.0)
integrator = hoomd.md.Integrator(dt=dt, methods=[langevin], \
                                forces = [fene, kratky, lj])

simulation.operations.integrator = integrator

We will write the trajectory as a GSD file. We specified a Periodic trigger to write the state to the trajectory file every multiple of a given number of iterations.

In [ ]:
gsd_writer = hoomd.write.GSD(filename='trajectory.gsd', trigger=hoomd.trigger.Periodic(idump), dynamic=['property', 'momentum'], mode='wb')
simulation.operations.writers.append(gsd_writer)

Finally, we are ready to run our simulation.

In [ ]:
simulation.run(itermax)

Let us check out the state we reached

In [ ]:
render(simulation.state.get_snapshot())

## Analysis

Read the trajectory:

In [ ]:
gsd_writer.flush()  # this is necessary sometimes
traj = gsd.hoomd.open('trajectory.gsd')

Let us read the time

In [ ]:
steps = np.array([traj[i].configuration.step for i in range(len(traj))])
times = steps * dt

Let us read the positions and velocities

In [ ]:
# we retain the index of the mobile particles only, i.e. index 0
mobile = traj[0].particles.typeid == 0

In [ ]:
positions = np.array([traj[i].particles.position[mobile] for i in range(len(traj))])
velocities = np.array([traj[i].particles.velocity[mobile] for i in range(len(traj))])

First of all, let us compute the radius of gyration of the polymer chain:
\begin{aligned}
R_g^2 = \frac{1}{N} \sum \limits_{i=1}^{N} \left( r_i - r_\text{mean} \right)^2
\end{aligned}

In [ ]:
positions_mean = np.mean(positions, axis=1).reshape(-1,1,3)
rg2 = np.mean(np.sum((positions-positions_mean)**2, axis=2), axis=1)

In [ ]:
plt.plot(times, rg2)
plt.xlabel('simulation time')
plt.ylabel('Rg2')
plt.show()

Thus we see that the radius of gyration progressively relax towards its equilibrium value

Let us now compute the mean thermal energy among the monomers:
\begin{aligned}
\langle v^2 \rangle = \frac{1}{N} \sum \limits_{i=1}^N (v_i^2 - v_\text{mean})
\end{aligned}

In [ ]:
v_mean = np.mean(velocities, axis=1).reshape(-1,1,3)
v_thermal = np.mean(np.sum((velocities-v_mean)**2, axis=2), axis=1)

In [ ]:
plt.plot(times, v_thermal)
plt.xlabel('simulation time')
plt.ylabel('thermal energy')
plt.show()

We see that after a transient regime due to the initial condition, the thermal energy per monomer oscillates around its expected mean value of $\langle \delta v^2 \rangle = 3 k_\text{B} T$.

## Make movie

<div class="alert alert-warning">
This may take time to complete depending on the number of time points in the trajectory. Use `traj[k0:k1:dk]` to adjust the start, end and number of time points in the trajectory.
</div>

In [ ]:
gsd_writer.flush()  # this is necessary sometimes
traj = gsd.hoomd.open('trajectory.gsd')

In [ ]:
render_movie(traj)